In [1]:
import torchvision as tv
import phototour
import torch
from tqdm import tqdm 
import numpy as np
import torch.nn as nn
import math 

lib_train = phototour.PhotoTour('.','liberty', download=True, train=True, mode = 'triplets', augment = True, nsamples=409600)
yos_train = phototour.PhotoTour('.','yosemite', download=True, train=True, mode = 'triplets', augment = True)
nd_train = phototour.PhotoTour('.','notredame', download=True, train=True, mode = 'triplets', augment = True)

eval_db = phototour.PhotoTour('.','yosemite', download=True, train=False)
# train_db = torch.utils.data.ConcatDataset((lib_train, yos_train))
train_db = nd_train
train_name = 'notredame'

# Found cached data ./liberty.pt
# Found cached data ./yosemite.pt
# Found cached data ./notredame.pt
# Found cached data ./yosemite.pt


In [2]:
import tfeat_model
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.SGD(tfeat.parameters(), lr=0.1, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,
                                             num_workers=32)

In [ ]:
fpr_per_epoch = []

for e in range(300):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n = data_n.unsqueeze(1).float().cuda()
        out_a, out_p, out_n = tfeat(data_a), tfeat(data_p), tfeat(data_n)
        loss = F.triplet_margin_loss(out_a, out_p, out_n, margin=2, swap=True) 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    #evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l,lbls.numpy()))
        d = np.hstack((d,dists))
        
    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l==1]
    dist_neg = d[l!=1]
    dist_pos,indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum())/dist_neg.numel()
    print(e,fpr95)
    fpr_per_epoch.append([e,fpr95])
    scheduler.step()
    np.savetxt('fpr.txt', np.array(fpr_per_epoch), delimiter=',') 
        


3334it [06:21,  8.75it/s]


0 0.1657


3334it [06:25,  8.65it/s]


1 0.12372


3334it [06:20,  8.76it/s]


...

3334it [06:21,  8.73it/s]


45 0.07304


3334it [06:20,  8.76it/s]


46 0.0667


1919it [03:43,  8.60it/s]

In [ ]:
torch.save(tfeat.state_dict(), train_name+'-tfeat.params')